In [6]:
import praw
import json
import pymongo
import urllib.request
from items import Redditor
import re
import pandas as pd
from multiprocessing import Pool
import threading
from queue import Queue

reddit = praw.Reddit('bot1', user_agent='bot1 user agent')

# Clean bot username text file

In [ ]:
path = '/Users/royroongseang/Documents/Uchicago MScA/Big Data/Reddit-Bot-Classifier'

In [37]:
with open(path+'/botaccounts/bot_accounts_2019.txt', 'r') as data:
    lines = data.readlines()

bot_usernames = []

for line in lines:
    bot_usernames.append(str.strip(line))

In [60]:
newbotlist = []
for name in bot_usernames:
    if name.startswith('u/'):
        newbotlist.append(name.split('u/')[1].split('\t')[0])
    else:
        newbotlist.append(name)

In [64]:
pd.DataFrame(newbotlist,columns=['bot_acct']).to_csv('bot_accounts2019.csv',encoding='utf-8',index=False)

In [69]:
bot_names = pd.read_csv(path+'/botaccounts/bot_accounts2019.csv')['bot_acct'].tolist()

# Function to Pull User Account Data

In [70]:
print_lock = threading.Lock()

# Scrape user information and store it in mongodb
def scrape_users(usernames, is_bot):
    counter = 0
    for author in usernames:

        redditor = Redditor()
        
        # try/except because some users have been deleted
        try:
            counter += 1
            user = reddit.redditor(author)

            redditor.username = author
            redditor.post_karma = user.link_karma
            redditor.comment_karma = user.comment_karma
            redditor.cake_day = user.created_utc
            redditor.is_bot = is_bot
            print('pass'+str(counter))

        except Exception as e:
            print(e)
            print("Reddit account " + author + " has been deleted")
            #record_deleted_username(author)
            continue


        insert_user_into_mongo_db(redditor)
        


def insert_user_into_mongo_db(redditor):
    client = pymongo.MongoClient('mongodb://localhost:27017')
    db = client['redditors']
    collection = db['redditors']
    collection.update({'username': redditor.username}, redditor.__dict__, upsert=True)
    client.close()



def scrape_all_users(bot_names):
    
    scrape_users(bot_names, True)
    #scrape_users(account_names, False)


# Scrape bot and user accounts and insert their data into the db
def scrape_accounts(bot_account_names):

    scrape_all_users(bot_account_names)

In [71]:
#Call function with using bot usernames
scrape_accounts(bot_names)

pass1


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:37: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.


pass2
pass3
pass4
pass5
pass6
pass7
pass8
pass9
pass10
pass11
pass12
pass13
pass14
pass15
pass16
pass17
pass18
pass19
pass20
pass21
pass22
pass23
pass24
pass25
pass26
pass27
pass28
pass29
pass30
pass31
pass32
pass33
pass34
pass35
pass36
pass37
pass38
pass39
pass40
pass41
pass42
pass43
pass44
pass45
pass46
pass47
pass48
pass49
pass50
pass51
pass52
pass53
pass54
pass55
pass56
pass57
pass58
pass59
pass60
pass61
pass62
pass63
pass64
pass65
pass66
pass67
pass68
pass69
pass70
pass71
pass72
pass73
pass74
pass75
pass76
pass77
pass78
pass79
pass80
pass81
pass82
pass83
pass84
pass85
pass86
pass87
pass88
pass89
pass90
pass91
pass92
pass93
pass94
pass95
pass96
pass97
pass98
pass99
pass100
pass101
pass102
pass103
pass104
pass105
pass106
pass107
pass108
pass109
pass110
pass111
pass112
pass113
pass114
pass115
pass116
pass117
pass118
pass119
pass120
pass121
pass122
pass123
pass124
pass125
pass126
pass127
pass128
pass129
pass130
pass131
pass132
pass133
pass134
pass135
pass136
pass137
pass138
pass139
pa

# Export user data from Mongo to csv

In [72]:
def _connect_mongo(host, port, username, password, db):
    """ A util for making a connection to mongo """

    if username and password:
        mongo_uri = 'mongodb://%s:%s@%s:%s/%s' % (username, password, host, port, db)
        conn = MongoClient(mongo_uri)
    else:
        conn = MongoClient(host, port)


    return conn[db]


def read_mongo(db, collection, query={}, host='localhost', port=27017, username=None, password=None, no_id=True):
    """ Read from Mongo and Store into DataFrame """

    # Connect to MongoDB
    db = _connect_mongo(host=host, port=port, username=username, password=password, db=db)

    # Make a query to the specific DB and Collection
    cursor = db[collection].find(query)

    # Expand the cursor and construct the DataFrame
    df =  pd.DataFrame(list(cursor))

    # Delete the _id
    if no_id:
        del df['_id']

    return df

In [73]:
user_data = read_mongo('redditors','redditors')

In [75]:
user_data.to_csv('bot_userdata.csv',encoding='utf-8',index=False)